In [1]:
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

import pandas as pd
import numpy as np

In [2]:
apps = pd.read_csv("https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/data/samples/app_samp.csv.gz?raw=true",
                   index_col=0,
                   compression="gzip")

Берем в работу две новых таблицы 

In [3]:
bureau = pd.read_csv("https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/data/samples/bur_samp.csv.gz?raw=true",
                     compression="gzip",
                     index_col=0)

In [4]:
bureau_balance = pd.read_csv("https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/data/samples/bur_bal_samp.csv.gz?raw=true",
                             compression="gzip",
                             index_col=0)

Немного общей статистики по таблице bureau

In [5]:
def get_stats(df):
    univ = pd.Series(data=[df[col].nunique()
                           if df[col].dtype == "object"
                           else np.nan
                           for col in df.columns],
                     index=df.columns)
    stats = pd.concat([df.dtypes,
                       univ,
                       df.isna().mean().round(4)],
                      axis=1)
    stats.columns = ["type", "univ", "pct_nan"]
    return stats

In [6]:
bureau_stats = get_stats(bureau)

print(bureau_stats)

                           type  univ  pct_nan
SK_ID_CURR                int64   NaN   0.0000
SK_ID_BUREAU              int64   NaN   0.0000
CREDIT_ACTIVE            object   3.0   0.0000
CREDIT_CURRENCY          object   2.0   0.0000
DAYS_CREDIT               int64   NaN   0.0000
CREDIT_DAY_OVERDUE        int64   NaN   0.0000
DAYS_CREDIT_ENDDATE     float64   NaN   0.0619
DAYS_ENDDATE_FACT       float64   NaN   0.3739
AMT_CREDIT_MAX_OVERDUE  float64   NaN   0.6564
CNT_CREDIT_PROLONG        int64   NaN   0.0000
AMT_CREDIT_SUM          float64   NaN   0.0000
AMT_CREDIT_SUM_DEBT     float64   NaN   0.1425
AMT_CREDIT_SUM_LIMIT    float64   NaN   0.3274
AMT_CREDIT_SUM_OVERDUE  float64   NaN   0.0000
CREDIT_TYPE              object   9.0   0.0000
DAYS_CREDIT_UPDATE        int64   NaN   0.0000
AMT_ANNUITY             float64   NaN   0.7544


Из таблицы bureau  мы сможем вытянуть "кредитную историю клиента". Здесь есть записи по текущим и закрытым кредитам, которые мы будем использовать для подсчета стат. показателей далее. Жаль, что AMT_ANNUITY имеет так много пропусков - расчитывал ее использовать для определения совокупной "долговой нагрузки" (отношение платежей по открытым кредитам к головому доходу).

Хуже дела обстоят с таблицей bureau_balance. Долго не мог найти ей достойного применения.

Во-первых, если я правильно понимаю, то *CREDIT_DAY_OVERDUE | Number of days past due on CB credit at the time of application for related loan in our sample* из bureau фактически дублирует *MONTHS_BALANCE | Month of balance relative to application date (-1 means the freshest balance date)...* из bureau_balance, т.е. вроде не несет новой информации

Во-вторых,в bureau_balance, все-таки не все кредиты из bureau.

В-третьх, как оказалось, две таблицы даже противоречат друг другу:

In [7]:
bureau.loc[bureau["SK_ID_BUREAU"] == 5009147, :]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
69163,327899,5009147,Active,currency 1,-367,0,1459.0,NaN,NaN,0,3159607.5,2820298.5,0.0,0.0,Consumer credit,-40,0.0


In [8]:
bureau_balance.loc[bureau_balance["SK_ID_BUREAU"] == 5009147, :]

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
5012541,5009147,0,C
5012542,5009147,-1,C
5012543,5009147,-2,C
5012544,5009147,-3,C
5012545,5009147,-4,C
5012546,5009147,-5,C
5012547,5009147,-6,C
5012548,5009147,-7,C
5012549,5009147,-8,0
5012550,5009147,-9,0


Судя из *bureau* кредит 5009147 еще не закрыт, а информация там обновлена за 40 дней до заявки в Home Credit, тогда как из *bureau_balance* следует, что кредит закрыт 7 мес. назад!?

В итоге решил использовать *bureau_balance* как источник актуализации информации по открытым кредитам в *bureau* (такой подход себя оправдал небольшим ростом метрики на тесте) 

In [9]:
bureau_mod = bureau.copy()

closed = (bureau_balance
          .groupby("SK_ID_BUREAU")
          .filter(lambda d: (d["STATUS"] == "C").any())["SK_ID_BUREAU"]
          .unique())

bureau_mod["CREDIT_ACTIVE"] = [v if v != "Active" or i not in closed
                               else "Closed" for v, i in
                               list(zip(bureau_mod["CREDIT_ACTIVE"],
                                        bureau_mod["SK_ID_BUREAU"]))]

In [10]:
(bureau_mod["DAYS_CREDIT_ENDDATE"]
 .fillna(bureau_mod["DAYS_ENDDATE_FACT"],
         inplace=True))

#(bureau_mod
# .dropna(subset=["DAYS_CREDIT_ENDDATE"],
#         inplace=True))

Собственно, из *bureau* расчитываем такие показатели по каждому клиенту:
* общее количество кредитов
* доля открытых кредитов
* доля просроченных кредитов
* сумма "непогашенных" кредитов, которая "висит" на клиентепо состоянию на дату заявки в Home Credit (отношение срока до конца конца кредита к общейпродолжительности кредита, умноженное на сумму кредита)

In [11]:
def get_loan_balance(df):
    out = df.loc[(df["CREDIT_ACTIVE"] == "Active") &
                 (df["DAYS_CREDIT_ENDDATE"] > 0), :].copy()

    return ((out["DAYS_CREDIT_ENDDATE"] /
             (out["DAYS_CREDIT"].abs() +
              out["DAYS_CREDIT_ENDDATE"]) *
             out["AMT_CREDIT_SUM"]).sum())


bureau_mod_gr = bureau_mod.groupby("SK_ID_CURR")

bureau_curr = pd.DataFrame(index=apps["SK_ID_CURR"])

bureau_curr["loan_num"] = bureau_mod_gr.size()

bureau_curr["loan_act"] = (bureau_mod_gr
                           .apply(lambda d:
                                  len(d.loc[d["CREDIT_ACTIVE"] == "Active",
                                            :]) / len(d)))

bureau_curr["loan_ovd"] = (bureau_mod_gr
                           .apply(lambda d:
                                  len(d.loc[d["CREDIT_DAY_OVERDUE"] != 0,
                                            :]) / len(d)))

bureau_curr["loan_bal"] = (bureau_mod_gr
                           .apply(get_loan_balance))

bureau_curr.fillna(0,
                   axis=0,
                   inplace=True)

Тренировочная и тестовая выборки по основной таблице из [ноутбука](https://github.com/woldemarg/ds_tests/blob/master/kaggle_solutions/home_credit_default_risk/scripts/notebooks/baseline_model.ipynb)  

In [12]:
X_train = pd.read_csv("https://media.githubusercontent.com/media/woldemarg/ds_tests/master/kaggle_solutions/home_credit_default_risk/derived/X_train.csv",
                      index_col=0)

X_test = pd.read_csv("https://media.githubusercontent.com/media/woldemarg/ds_tests/master/kaggle_solutions/home_credit_default_risk/derived/X_test.csv",
                     index_col=0)

y_train = pd.read_csv("https://media.githubusercontent.com/media/woldemarg/ds_tests/master/kaggle_solutions/home_credit_default_risk/derived/y_train.csv",
                      index_col=0)

y_test = pd.read_csv("https://media.githubusercontent.com/media/woldemarg/ds_tests/master/kaggle_solutions/home_credit_default_risk/derived/y_test.csv",
                     index_col=0)

In [13]:
X_train_plus = X_train.join(bureau_curr)
X_test_plus = X_test.join(bureau_curr)

target = y_train["TARGET"].value_counts()
spw = target[0] / target[1]

xgb_model = XGBClassifier(random_state=1234,
                          objective="binary:logistic",
                          scale_pos_weight=spw,
                          n_jobs=-1)

xgb_model.fit(X_train_plus, y_train["TARGET"])

y_pred = xgb_model.predict(X_test_plus)

print("ROC-AUC on test: {}".format(roc_auc_score(y_test, y_pred)))


ROC-AUC on test: 0.6484098939929329


За счет добавления новых предикторов (4):
* метрика на тесте - 0,65
* метрика на сv - 0,69
* метрика на kaggle - 